In [1]:
import re
import pandas as pd

In [2]:
with open('_toc.yml', 'r') as infile:
    text = infile.read()

In [4]:
md_fns = re.findall('(Volumes.*?)\n', text)
md_fns = [fn.strip() for fn in md_fns]

In [10]:
!pwd

/Users/nealcaren/Documents/GitHub/fightordie


In [12]:
with open('_build/html/sitemap.txt', 'w') as outfile:
    for fn in md_fns:
        s = 'https://www.dareyoufight.org/' + fn + '\n'
        outfile.write(s)

In [4]:
meta = []
for md_fn in md_fns:
    with open(md_fn, 'r') as infile:
        md = infile.read()
        
    meta_text = md.split('-->')[0].split('<!--')[-1].splitlines()
    meta_d = {'fn': md_fn}
    for row in meta_text:
        if ':' in row:
            p = row.split(':')
            
            k = p[0]
            v = ':'.join(p[1:])
            
            k,v = k.strip(), v.strip()
            meta_d[k] = v
        if 'author' not in meta_d:
            meta_d['author'] = 'W.E.B. Du Bois'
        if 'journal' not in meta_d:
            meta_d['journal'] = 'The Crisis'        
    
    meta.append(meta_d)

In [5]:
df = pd.DataFrame(meta)

def title_link(row):
    link = 'https://www.dareyoufight.org/' + row['fn'].replace('.md','.html')
    title = row['title']
    t = f'[{title}]({link})'
    return t

df['Title'] = df.apply(title_link, axis=1)

In [6]:
df['Year'] = df['year']
df['Volume'] = df['volume']
df['Issue'] = df['issue'].str.replace('-Supplement','S')

keep = ['Year', 'Volume', 'Issue', 'Title']

md_table = df[keep].sort_values(by=['Year', 'Volume', 'Issue']).to_markdown(index=False)

In [7]:
t = f'''# Chronological Listing

{md_table}
'''

In [8]:
with open('chronological.md', 'w') as outfile:
    outfile.write(t)

In [9]:
with open('index_with_var.md', 'r') as infile:
    md = infile.read()

count = len(df)
md = md.replace('{COUNT}', f'{count}')

with open('index.md', 'w') as outfile:
    outfile.write(md)

In [10]:
print(count)

567
